In [12]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
data = [
    ('James', 34, 'M', 3000, 1),
    ('Anna', 28, 'F', 4100, 2),
    ('Lee', 23, 'M', 2700, 1)
]
columns = ['Name', 'Age', 'Gender', 'Salary', 'DeptId']
df = spark.createDataFrame(data, schema=columns)
df.createOrReplaceTempView('employees')
spark.sql('SELECT * FROM employees').show()
spark.sql('SELECT Name, Age FROM employees WHERE Salary > 3000').show()
spark.sql('SELECT AVG(Salary) FROM employees').show()


+-----+---+------+------+------+
| Name|Age|Gender|Salary|DeptId|
+-----+---+------+------+------+
|James| 34|     M|  3000|     1|
| Anna| 28|     F|  4100|     2|
|  Lee| 23|     M|  2700|     1|
+-----+---+------+------+------+

+----+---+
|Name|Age|
+----+---+
|Anna| 28|
+----+---+

+------------------+
|       avg(Salary)|
+------------------+
|3266.6666666666665|
+------------------+



In [13]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark.sql('''
SELECT Gender, SUM(Salary) as TotalSalary, Age
FROM employees
GROUP BY Gender, Age
ORDER BY Age
''').show()


+------+-----------+---+
|Gender|TotalSalary|Age|
+------+-----------+---+
|     M|       2700| 23|
|     F|       4100| 28|
|     M|       3000| 34|
+------+-----------+---+



In [14]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark.sql('''
SELECT Name, Age, Salary, ROW_NUMBER() OVER (PARTITION BY Age ORDER BY Salary DESC) as rank
FROM employees
''').show()


+-----+---+------+----+
| Name|Age|Salary|rank|
+-----+---+------+----+
|  Lee| 23|  2700|   1|
| Anna| 28|  4100|   1|
|James| 34|  3000|   1|
+-----+---+------+----+



In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Pertemuan9').getOrCreate()

# Data setup for complex SQL queries
employees = spark.createDataFrame([
    ('James', 34, 'M', 3000, 1),
    ('Anna', 28, 'F', 4100, 2),
    ('Lee', 23, 'M', 2700, 1)
], ['Name', 'Age', 'Gender', 'Salary', 'DeptId'])
departments = spark.createDataFrame([
    (1, 'HR'),
    (2, 'Marketing')
], ['DeptId', 'DeptName'])
projects = spark.createDataFrame([
    (1, 'Project A'),
    (2, 'Project B')
], ['DeptId', 'ProjectName'])
employees.createOrReplaceTempView('employees')
departments.createOrReplaceTempView('departments')
projects.createOrReplaceTempView('projects')

# Complex SQL query involving multiple joins and subqueries
spark.sql('''
SELECT e.Name, e.Age, d.DeptName, p.ProjectName
FROM employees e
JOIN departments d ON e.DeptId = d.DeptId
JOIN projects p ON e.DeptId = p.DeptId
''').show()


[Stage 1:>                  (0 + 3) / 4][Stage 2:==============>    (3 + 1) / 4]

+-----+---+---------+-----------+
| Name|Age| DeptName|ProjectName|
+-----+---+---------+-----------+
|  Lee| 23|       HR|  Project A|
|James| 34|       HR|  Project A|
| Anna| 28|Marketing|  Project B|
+-----+---+---------+-----------+



In [8]:
spark.sql('''
SELECT d.DeptName, AVG(e.Salary) AS AverageSalary
FROM employees e
JOIN departments d ON e.DeptId = d.DeptId
GROUP BY d.DeptName
''').show()

spark.sql('''
SELECT e1.Name, d.DeptName, e1.Salary,
       COUNT(e2.Salary) + 1 AS Rank
FROM employees e1
JOIN departments d ON e1.DeptId = d.DeptId
LEFT JOIN employees e2
    ON e1.DeptId = e2.DeptId AND e2.Salary > e1.Salary
GROUP BY e1.Name, d.DeptName, e1.Salary
ORDER BY d.DeptName, Rank
''').show()

+---------+-------------+
| DeptName|AverageSalary|
+---------+-------------+
|       HR|       2850.0|
|Marketing|       4100.0|
+---------+-------------+

+-----+---------+------+----+
| Name| DeptName|Salary|Rank|
+-----+---------+------+----+
|James|       HR|  3000|   1|
|  Lee|       HR|  2700|   2|
| Anna|Marketing|  4100|   1|
+-----+---------+------+----+



In [9]:
spark.sql('''
SELECT d.DeptName, e.Name, e.Age, e.Salary,
       AVG(e.Salary) OVER (PARTITION BY d.DeptName ORDER BY e.Age) AS TrendGaji
FROM employees e
JOIN departments d ON e.DeptId = d.DeptId
ORDER BY d.DeptName, e.Age
''').show()

+---------+-----+---+------+---------+
| DeptName| Name|Age|Salary|TrendGaji|
+---------+-----+---+------+---------+
|       HR|  Lee| 23|  2700|   2700.0|
|       HR|James| 34|  3000|   2850.0|
|Marketing| Anna| 28|  4100|   4100.0|
+---------+-----+---+------+---------+



### 6. Homework
- **Tugas 1**: Gunakan Spark SQL untuk mencari total gaji dan jumlah karyawan per departemen. Buat visualisasi perbandingan antar departemen.
- **Tugas 2**: Temukan karyawan dengan gaji di atas rata-rata dalam setiap kelompok usia dan visualisasikan data ini dalam bentuk grafik batang atau pie chart.
- **Tugas 3**: Buat dataset yang lebih besar (misalnya, 100+ baris) dan lakukan analisis mendalam menggunakan SQL functions seperti `SUM()`, `AVG()`, `COUNT()`, serta `JOIN` antar tabel serta buat visualisasi yang menarik.
